## Implementeer elk gemaakt ETL-schema in Python

**imports & connection**

In [27]:
import pandas as pd
import pyodbc
from datetime import datetime

ModuleNotFoundError: No module named 'ace_tools'

In [6]:
DB = {'servername': 'LAPTOP-LPE28RPE\SQLEXPRESS', 
    'database': 'United_outdoors'}

export_conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + DB['servername'] + 
                              ';DATABASE=' + DB['database'])

export_cursor = export_conn.cursor()
export_cursor   

**Database connection**

In [13]:

Adventure = {
    'servername' : 'LAPTOP-LPE28RPE\SQLEXPRESS',
    'database' : 'AdventureWorks2019'
}


Northwind = {
    'servername' : 'LAPTOP-LPE28RPE\SQLEXPRESS',
    'database' : 'Northwind'
}

access_db_path = r'C:\Users\Humberto de Castro\OneDrive\Desktop\SEM4\AenC\aenc.accdb'

#Connect to AdventureWorks
Adventure_conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={Adventure['servername']};DATABASE={Adventure['database']};Trusted_Connection=yes;")
Adventure_cursor = Adventure_conn.cursor()

#Connect to Northwind
Northwind_conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={Northwind['servername']};DATABASE={Northwind['database']};Trusted_Connection=yes;")
Northwind_cursor = Northwind_conn.cursor()

#Connect to AenC
AenC_conn = pyodbc.connect(f"DRIVER={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={access_db_path};")



In [25]:
AdventureWorks_queries = {
    'SalesOrderHeader' : 'SELECT * FROM Sales.SalesOrderHeader',
    'SalesOrderDetail' : 'SELECT * FROM Sales.SalesOrderDetail'
}

Northwind_queries = {
    'Orders' : 'SELECT * FROM Orders',
    'OrderDetails' : 'SELECT * FROM [Order Details]'
}

AenC_queries = {
    'sales_order' : 'SELECT * FROM sales_order',
    'sales_order_item' : 'SELECT * FROM sales_order_item'
}

dataframes = {}

# Lees elke tabel in een DataFrame

for table_name, query in AdventureWorks_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, Adventure_conn)

for table_name, query in Northwind_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, Northwind_conn)

for table_name, query in AenC_queries.items():
    dataframes[table_name] = pd.read_sql_query(query, AenC_conn)

#als je ik elk tabel als een dataframe/ variabele wil behandelen of aanroepen moet ik dit uitvoeren.
for table_name, df in dataframes.items():
    globals()[table_name] = df


C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_7484\1723358151.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframes[table_name] = pd.read_sql_query(query, Adventure_conn)
C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_7484\1723358151.py:24: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframes[table_name] = pd.read_sql_query(query, Northwind_conn)
C:\Users\Humberto de Castro\AppData\Local\Temp\ipykernel_7484\1723358151.py:27: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dataframes[table_na

### Order

**Extract**

In [26]:
#eerst horizontaal samenvoegen per database:
AdventureWorks_Order = pd.merge(SalesOrderHeader, SalesOrderDetail, on = 'SalesOrderID')
Northwind_Order = pd.merge(Orders, OrderDetails, on = 'OrderID')
AenC_Order = pd.merge(sales_order, sales_order_item, on = 'id')

**Transform**

**Check of orderID's conflicten**

In [29]:
AdventureWorks_Order.rename(columns={'SalesOrderID': 'OrderID'}, inplace=True)

order_order_ids = set(AdventureWorks_Order['OrderID'])
order_details_order_ids = set(Northwind_Order['OrderID'])
Order_AenC_ids = set(AenC_Order['id'])

# Zoek conflicten tussen AdventureWorks en Northwind
conflicting_order_ids_1 = order_order_ids.intersection(order_details_order_ids)

# Zoek conflicten tussen AdventureWorks en AenC
conflicting_order_ids_2 = order_order_ids.intersection(Order_AenC_ids)

# Zoek conflicten tussen Northwind en AenC
conflicting_order_ids_3 = order_details_order_ids.intersection(Order_AenC_ids)

# Combineer alle conflicterende ID's
all_conflicting_order_ids = conflicting_order_ids_1.union(conflicting_order_ids_2).union(conflicting_order_ids_3)

if all_conflicting_order_ids:
    print("Conflicting Order IDs found:", all_conflicting_order_ids)
else:
    print("No conflicts found in Order IDs.")

# Display the conflicting Order IDs (if any)
conflicts_df = pd.DataFrame(list(all_conflicting_order_ids), columns=['Conflicting Order IDs'])
print(conflicts_df)

No conflicts found in Order IDs.
Empty DataFrame
Columns: [Conflicting Order IDs]
Index: []


In [ ]:
# Zorg dat alle kolommen etc matchen voordat je ze verticaal gaat samenvoegen
AdventureWorks_Order = AdventureWorks_Order.loc[:, ['SalesOrderID', 'SalesOrderDetailID', 'OrderQty', 'UnitPrice', 'UnitPriceDiscount', 'Order_Date', 'CustomerID', 'EmployeeID', 'ShipMethodID', 'ProductID']]

Northwind_Order = Northwind_Order.loc[:, ['OrderID', 'Quantity', 'UnitPrice', 'Discount', 'OrderDate', 'CustomerID', 'EmployeeID', 'ShipMethodID', 'ShipVia', 'ProductID']]

